In [5]:
import sys
print(sys.executable)


C:\Python311\python.exe


In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# Set your data directories
DATA_DIR = Path(r"D:\Dev\SIH\Database\fish-platform\data")
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

RAW_FILE = RAW_DIR / "occurrences.csv"   # your raw CSV
CLEAN_FILE = PROCESSED_DIR / "occurrences_clean.csv"
QA_REPORT = PROCESSED_DIR / "cleaning_report.json"


In [8]:
df = pd.read_csv(RAW_FILE, low_memory=False)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist()[:50])
display(df.head(3))

# Missingness
missing = df.isna().mean().sort_values(ascending=False).head(40)
display(pd.DataFrame({"missing_fraction": missing}))


Shape: (1167, 133)
Columns: ['species', 'basisOfRecord', 'brackish', 'catalogNumber', 'class', 'classid', 'date_end', 'date_mid', 'date_start', 'date_year', 'decimalLatitude', 'decimalLongitude', 'eventDate', 'family', 'familyid', 'genus', 'genusid', 'gigaclass', 'gigaclassid', 'infraphylum', 'infraphylumid', 'institutionCode', 'kingdom', 'kingdomid', 'locality', 'marine', 'occurrenceID', 'occurrenceStatus', 'order', 'orderid', 'ownerInstitutionCode', 'parvphylum', 'parvphylumid', 'phylum', 'phylumid', 'references', 'scientificName', 'scientificNameID', 'speciesid', 'subphylum', 'subphylumid', 'superclass', 'superclassid', 'id', 'dataset_id', 'node_id', 'dropped', 'absence', 'originalScientificName', 'aphiaID']


,species,basisOfRecord,brackish,catalogNumber,class,classid,date_end,date_mid,date_start,date_year,...,sex,coordinatePrecision,occurrenceRemarks,subfamily,subfamilyid,organismQuantity,organismQuantityType,vernacularName,endDayOfYear,footprintSRS
0,Sardinella longiceps,HumanObservation,False,4012,Teleostei,293496.0,-3.865795e+12,-3.865795e+12,-3.865795e+12,1847.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sardinella longiceps,HumanObservation,False,NaN,Teleostei,293496.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sardinella longiceps,PreservedSpecimen,False,IO/SS/FIS/00089,Teleostei,293496.0,1.276474e+12,1.276474e+12,1.276474e+12,2010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,missing_fraction
previousIdentifications,0.999143
identificationID,0.999143
locationID,0.999143
countryCode,0.999143
acceptedNameUsageID,0.999143
verbatimCoordinateSystem,0.999143
footprintSRS,0.999143
disposition,0.999143
namePublishedInID,0.999143
verbatimCoordinates,0.999143


In [9]:
drop_cols = [
    "institutionCode","ownerInstitutionCode","collectionCode","collectionID",
    "datasetID","dataset_id","datasetName","institutionID","node_id","id",
    "recordNumber","recordedBy","identifiedBy","eventID","identificationID",
    "disposition","preparations","previousIdentifications","associatedReferences",
    "bibliographicCitation","references","gigaclass","gigaclassid","infraphylum",
    "infraphylumid","parvphylum","parvphylumid","subphylum","subphylumid",
    "superclass","superclassid","subfamily","subfamilyid","absence","dropped",
    "flags","verbatimCoordinates","verbatimLatitude","verbatimLongitude",
    "verbatimSRS","verbatimCoordinateSystem","license","accessRights",
    "rightsHolder","footprintWKT","footprintSRS"
]

drop_existing = [c for c in drop_cols if c in df.columns]
print("Dropping columns:", drop_existing)
df = df.drop(columns=drop_existing, errors="ignore")
print("New shape:", df.shape)


Dropping columns: ['institutionCode', 'ownerInstitutionCode', 'collectionCode', 'collectionID', 'datasetID', 'dataset_id', 'datasetName', 'institutionID', 'node_id', 'id', 'recordNumber', 'recordedBy', 'identifiedBy', 'eventID', 'identificationID', 'disposition', 'preparations', 'previousIdentifications', 'associatedReferences', 'bibliographicCitation', 'references', 'gigaclass', 'gigaclassid', 'infraphylum', 'infraphylumid', 'parvphylum', 'parvphylumid', 'subphylum', 'subphylumid', 'superclass', 'superclassid', 'subfamily', 'subfamilyid', 'absence', 'dropped', 'flags', 'verbatimCoordinates', 'verbatimLatitude', 'verbatimLongitude', 'verbatimSRS', 'verbatimCoordinateSystem', 'license', 'accessRights', 'rightsHolder', 'footprintWKT', 'footprintSRS']
New shape: (1167, 87)


In [10]:
# Convert depth and environmental columns to numeric
num_cols = ["decimalLatitude","decimalLongitude","depth","minimumDepthInMeters",
            "maximumDepthInMeters","coordinateUncertaintyInMeters",
            "individualCount","sst","sss"]

for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].replace(['NO_DEPTH','NO_DATA',''], np.nan), errors='coerce')

# Parse eventDate
if "eventDate" in df.columns:
    df["eventDate"] = pd.to_datetime(df["eventDate"], errors='coerce')
    df["year"] = df["eventDate"].dt.year
    df["month"] = df["eventDate"].dt.month
    df["day"] = df["eventDate"].dt.day

display(df[["scientificName","decimalLatitude","decimalLongitude","eventDate","year","sst","sss","depth"]].head())


,scientificName,decimalLatitude,decimalLongitude,eventDate,year,sst,sss,depth
0,Sardinella longiceps,10.000000,93.000000,1847-07-02 12:00:00+00:00,1847.0,28.91,32.43,NaN
1,Sardinella longiceps,12.410531,53.267271,NaT,NaN,26.50,35.92,NaN
2,Sardinella longiceps,8.660000,76.560000,NaT,NaN,28.23,34.35,25.0
3,Sardinella longiceps,8.470000,76.490000,NaT,NaN,28.36,34.36,100.0
4,Sardinella longiceps,23.833300,57.666700,NaT,NaN,28.29,36.85,NaN


In [11]:
essential = ["scientificName","decimalLatitude","decimalLongitude"]
present_essential = [c for c in essential if c in df.columns]

before = len(df)
df = df.dropna(subset=present_essential, how='any')
df = df[(df.decimalLatitude.between(-90,90)) & (df.decimalLongitude.between(-180,180))]
print(f"Dropped {before - len(df)} rows. Remaining: {len(df)}")


Dropped 0 rows. Remaining: 1167


In [12]:
# Remove exact duplicates
subset_keys = ["scientificName","eventDate","decimalLatitude","decimalLongitude"]
subset_keys = [c for c in subset_keys if c in df.columns]
before = len(df)
df = df.drop_duplicates(subset=subset_keys)
print(f"Dropped {before - len(df)} duplicate rows. Remaining: {len(df)}")


Dropped 243 duplicate rows. Remaining: 924


In [13]:
if "sst" in df.columns:
    df["sst_bad"] = ~df["sst"].between(-2,40)
if "sss" in df.columns:
    df["sss_bad"] = ~df["sss"].between(0,50)

print("Rows with unrealistic SST:", df["sst_bad"].sum() if "sst_bad" in df.columns else 0)
print("Rows with unrealistic SSS:", df["sss_bad"].sum() if "sss_bad" in df.columns else 0)


Rows with unrealistic SST: 11
Rows with unrealistic SSS: 11


In [14]:
report = {
    "original_rows": int(before),
    "cleaned_rows": int(len(df)),
    "columns": df.columns.tolist(),
    "num_species": int(df["scientificName"].nunique()),
}

with open(QA_REPORT, "w") as f:
    json.dump(report, f, indent=2)

df.to_csv(CLEAN_FILE, index=False)
print("Saved cleaned CSV:", CLEAN_FILE)
print("Saved QA report:", QA_REPORT)


Saved cleaned CSV: D:\Dev\SIH\Database\fish-platform\data\processed\occurrences_clean.csv
Saved QA report: D:\Dev\SIH\Database\fish-platform\data\processed\cleaning_report.json
